In [1]:
import os
import glob
import h5py
import pickle
import shutil
from itertools import chain

import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from scipy import signal, ndimage

import caiman as cm
import caiman.paths
from caiman.source_extraction.volpy import utils
from caiman.source_extraction.volpy.volparams import volparams
from caiman.source_extraction.volpy.volpy import VOLPY
from caiman.source_extraction.volpy.spikepursuit import signal_filter

2023-05-12 23:47:00.345633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
weights = None                                # if None, use ROIs for initialization; to reuse weights check reuse weights block 

fr = 500 
template_size = 0.01                          # half size of the window length for spike templates, default is 20 ms 
context_size = 2                          # number of pixels surrounding the ROI to censor from the background PCA
visualize_ROI = False                         # whether to visualize the region of interest inside the context region
flip_signal = True                            # Important!! Flip signal or not, True for Voltron indicator, False for others
hp_freq_pb = 1 / 3                            # parameter for high-pass filter to remove photobleaching
hp_freq = 5
clip = 100                                    # maximum number of spikes to form spike template
threshold_method = 'adaptive_threshold'       # adaptive_threshold or simple 
min_spikes= 5                                # minimal spikes to be found  # was 50 for Jiannis' data
pnorm = 0.5                                   # a variable deciding the amount of spikes chosen for adaptive threshold method
desired_fp = 10**(-4)  # note: was hard-coded for current use case
threshold = 3                                 # threshold for finding spikes only used in simple threshold method, Increase the threshold to find less spikes
do_plot = False                               # plot detail of spikes, template for the last iteration
ridge_bg= 0.01                                # ridge regression regularizer strength for background removement, larger value specifies stronger regularization 
sub_freq = 20                                 # frequency for subthreshold extraction
weight_update = 'ridge'                       # ridge or NMF for weight update
n_iter = 2

In [3]:
def cut_img(img):
    '''
    Cut initial 600 frames
    '''
    return img[600:]

In [4]:
mouse_dir = '/Volumes/CLab/sungmoo/Real analysis_traces flattened'

pre_process = True
remove_moving_part = True  # True for Jiannis data, False for Bruker data
mouse_list = [glob.glob(os.path.join(mouse_dir, starter+'*')) for starter in['F9', 'M9']]
mouse_list = list(chain(*mouse_list))

In [5]:
mouse_list

['/Volumes/CLab/sungmoo/Real analysis_traces flattened/F957L',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F99xR',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F986L',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F960R',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F975R',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F991R',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F954R',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F981R',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F993L',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/F976L',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/M956R',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/M955L',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/M958R',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/M982L',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/M984L',
 '/Volumes/CLab/sungmoo/Real analysis_traces flattened/

In [ ]:

for mouse in mouse_list:
    print(mouse)
    working_dir = mouse
    img_list = glob.glob(os.path.join(working_dir, '*.tif'))

    save_dir = os.path.join(working_dir, 'volpy_results')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    fail_dir = os.path.join(working_dir, 'failed')
    if not os.path.exists(fail_dir):
        os.makedirs(fail_dir)

    for img_path in img_list:
        ## read image and ROI
        img = io.imread(img_path)
        if pre_process:
            img = cut_img(img)
        mean_img = img.mean(axis=0)
        ROI_path = img_path[:-4] + '_mask.h5'  # only for jiannis data
        img_id = os.path.basename(img_path).replace('.tif', '')
        print(img_id)
        with h5py.File(ROI_path, 'r') as fl:
            ROI = fl['cell_mask'][()]
        if remove_moving_part:
            solid = img.min(axis=0)
            solid = solid > 1.0
            # erode solid mask
            #solid = ndimage.binary_erosion(solid, iterations=1)
            ROI = ROI * solid.T
        #img = remove_moving_frames(img, ROI)
        T, d1, d2 = img.shape
        img_reshape = img.reshape(T, d1*d2, order='F')
        ## generating memory mapping
        mmap_path = caiman.paths.memmap_frames_filename(img_id[:15], [d1, d2], T, 'C')
        mmap_path = os.path.join(working_dir, mmap_path)
        fp = np.memmap(mmap_path, dtype='float32', mode='w+', shape=(d1*d2, T), order='C')
        fp[:] = img_reshape[:].T
        fp.flush()
        del fp
        ## assembly parameters for volpy
        ROIs = np.expand_dims(ROI.T, axis=0)
        index = list(range(len(ROIs)))     # index of ROIs to be used for spike extraction
        opts_dict={'fnames': mmap_path,
                'ROIs': ROIs,
                'fr': fr,
                'index': index,
                'weights': weights,
                'template_size': template_size, 
                'context_size': context_size,
                'visualize_ROI': visualize_ROI, 
                'flip_signal': flip_signal,
                'hp_freq': hp_freq,
                'hp_freq_pb': hp_freq_pb,
                'clip': clip,
                'threshold_method': threshold_method,
                'min_spikes':min_spikes,
                'pnorm': pnorm,
                #'desired_fp': desired_fp, 
                'threshold': threshold,
                'do_plot':do_plot,
                'ridge_bg':ridge_bg,
                'sub_freq': sub_freq,
                'weight_update': weight_update,
                'n_iter': n_iter}

        opts = volparams(params_dict=opts_dict)
        ## run volpy
        if 'dview' in locals():
            cm.stop_server(dview=dview)
        c, dview, n_processes = cm.cluster.setup_cluster(
            backend='local', n_processes=None, single_thread=False)
        try:
            vpy = VOLPY(n_processes=n_processes, dview=dview, params=opts)
            vpy.fit(n_processes=n_processes, dview=dview)
            ## visualize and save results
            print(np.where(vpy.estimates['locality'])[0])    # neurons that pass locality test
            idx = np.where(vpy.estimates['locality'] > 0)[0]
            utils.view_components(vpy.estimates, mean_img, [0], save_path=os.path.join(save_dir, img_id + '_summary.png'))
            with open(os.path.join(save_dir, img_id + '_volpy.pkl'), 'wb') as f:
                pickle.dump(vpy.estimates, f)
        except:
            shutil.move(src=img_path, dst=os.path.join(working_dir, 'failed'))